In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [16]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, 
                    columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'],axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])
target.shape

(506, 1)

In [41]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RN_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1,11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [18]:
features.mean()

CRIM         3.613524
ZN          11.363636
CHAS         0.069170
NOX          0.554695
RM           6.284634
DIS          3.795043
RAD          9.549407
TAX        408.237154
PTRATIO     18.455534
B          356.674032
LSTAT       12.653063
dtype: float64

In [19]:
type(features.mean()), type(features.mean().values)

(pandas.core.series.Series, numpy.ndarray)

In [21]:
features.mean().values.reshape(1,11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [31]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Calculate the MSE and RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)


In [65]:
def get_log_estimate(nr_rooms,
                    students_per_classroom,
                    next_to_river=False,
                    high_confidence=True):
    
    # Configure property
    property_stats[0][RN_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + 1*RMSE
        lower_bound = log_estimate - 1*RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [68]:
get_log_estimate(7, 15, next_to_river=True)

(3.3265788671351433, 3.701603137529404, 2.9515545967408827, 95)

In [69]:
np.median(boston_dataset.target)

21.2

In [77]:
# convert the log prices est using 1970s prices,
# as well as upper and lower bound into toda's prices

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE/np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                              next_to_river=False,
                                              high_confidence=False)
# convert to today's $$
dollar_est = np.e**log_est*1000*SCALE_FACTOR
dollar_hi = np.e**upper*1000*SCALE_FACTOR
dollar_low = np.e**lower*1000*SCALE_FACTOR


# round the dollar value to the neareset $000
rounded_est = np.around(dollar_est, -3)
rounded_high = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated propery value is {rounded_est}.')
print(f'At {conf}% range the valuation range is:')
print(f'USD {rounded_low} at the lower end and {rounded_high} at the higher end')



The estimated propery value is 827000.0.
At 68% range the valuation range is:
USD 685000.0 at the lower end and 997000.0 at the higher end


In [96]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True if the property is nexet to the river; false otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval
    """
    if rm < 1 or ptratio <1:
        print('That is unrealistic. Try again')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, students_per_classroom=ptratio,
                                              next_to_river=chas,
                                              high_confidence=large_range)
    # convert to today's $$
    dollar_est = np.e**log_est*1000*SCALE_FACTOR
    dollar_hi = np.e**upper*1000*SCALE_FACTOR
    dollar_low = np.e**lower*1000*SCALE_FACTOR


    # round the dollar value to the neareset $000
    rounded_est = np.around(dollar_est, -3)
    rounded_high = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated propery value is {rounded_est}.')
    print(f'At {conf}% range the valuation range is:')
    print(f'USD {rounded_low} at the lower end and {rounded_high} at the higher end')
    

In [97]:
get_dollar_estimate(rm=3, ptratio=12, chas=True)

The estimated propery value is 596000.0.
At 95% range the valuation range is:
USD 410000.0 at the lower end and 868000.0 at the higher end


In [99]:
import boston_valuation as val
val.get_dollar_estimate(6,12,True)

The estimated propery value is 783000.0.
At 95% range the valuation range is:
USD 538000.0 at the lower end and 1139000.0 at the higher end
